# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-24-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-24-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-07-25 04:47:39,34.223334,-82.461707,260,2,0,258.0,"Abbeville, South Carolina, US",1060.056265,0.769231
1,22001.0,Acadia,Louisiana,US,2020-07-25 04:47:39,30.295065,-92.414197,2111,62,0,2049.0,"Acadia, Louisiana, US",3402.369248,2.936997
2,51001.0,Accomack,Virginia,US,2020-07-25 04:47:39,37.767072,-75.632346,1059,15,0,1044.0,"Accomack, Virginia, US",3277.014482,1.416431
3,16001.0,Ada,Idaho,US,2020-07-25 04:47:39,43.452658,-116.241552,6732,45,0,6687.0,"Ada, Idaho, US",1397.878265,0.668449
4,19001.0,Adair,Iowa,US,2020-07-25 04:47:39,41.330756,-94.471059,20,0,0,20.0,"Adair, Iowa, US",279.642058,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,34994,35070,35229,35301,35475,35526,35615,35727,35928,35981
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3752,3851,3906,4008,4090,4171,4290,4358,4466,4570
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,20770,21355,21948,22549,23084,23691,24278,24872,25484,26159


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1094,1113,1147,1164,1181,1185,1186,1190,1211,1225
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,101,104,107,111,112,113,117,120,123,128
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1040,1052,1057,1068,1078,1087,1100,1111,1124,1136


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,22456,22824,23151,23273,23634,23741,23741,23924,24550,24602
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2091,2137,2214,2264,2311,2352,2397,2463,2523,2608
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,14792,15107,15430,15744,16051,16400,16646,16983,17369,17369


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,313,314,314,314,314,319,319,330,337,337
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,36,37,37,37,37,38,38,38,38,38


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
1678,1083.0,Limestone,Alabama,US,2020-07-25 04:47:39,34.811856,-86.983101,970,9,0,961.0,"Limestone, Alabama, US",980.639943,0.927835
1122,1063.0,Greene,Alabama,US,2020-07-25 04:47:39,32.855042,-87.956840,231,10,0,221.0,"Greene, Alabama, US",2847.984219,4.329004
484,1019.0,Cherokee,Alabama,US,2020-07-25 04:47:39,34.178060,-85.606390,191,7,0,184.0,"Cherokee, Alabama, US",729.118949,3.664921


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,4112531,145546,1261624
Brazil,2287475,85238,1693214
India,1337024,31358,849432


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,4112531,145546,1261624,2705361,2020-07-25 04:47:39,37.894113,-91.288292
Brazil,2287475,85238,1693214,509023,2020-07-25 04:47:39,-12.669522,-48.480493
India,1337024,31358,849432,456234,2020-07-25 04:47:39,23.356511,82.081064
Russia,799499,13026,587728,198745,2020-07-25 04:47:39,54.546312,62.120860
South Africa,421996,6343,245771,169882,2020-07-25 04:47:39,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,440185,8337,0
New York,410450,32596,0
Florida,402312,5653,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,440185,8337,0,431848,2020-07-25 04:47:39,37.778248,-120.728666
New York,410450,32596,0,377854,2020-07-25 04:47:39,42.671593,-75.579694
Florida,402312,5653,0,396659,2020-07-25 04:47:39,28.940755,-82.700744
Texas,380364,4790,0,375574,2020-07-25 04:47:39,31.647790,-98.615963
New Jersey,178345,15765,0,162580,2020-07-25 04:47:39,40.286441,-74.622450


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,223192,23465,0
California,Los Angeles,168935,4300,0
Arizona,Maricopa,104613,1718,0
Illinois,Cook,101411,4822,0
Florida,Miami-Dade,98430,1370,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,223192,23465,0,199727,2020-07-25 04:47:39,40.767273,-73.971526,36061.0
California,Los Angeles,168935,4300,0,164635,2020-07-25 04:47:39,34.308284,-118.228241,6037.0
Arizona,Maricopa,104613,1718,0,102895,2020-07-25 04:47:39,33.348359,-112.491815,4013.0
Illinois,Cook,101411,4822,0,96589,2020-07-25 04:47:39,41.841448,-87.816588,17031.0
Florida,Miami-Dade,98430,1370,0,97060,2020-07-25 04:47:39,25.611236,-80.551706,12086.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-22,35727,4358,24872,889,812,76,141900,35693,13302,19929,...,297952,1117,18379,13164,408,9398,10,1640,3583,2034
2020-07-23,35928,4466,25484,889,851,76,148027,36162,13595,20099,...,298731,1141,18986,13613,412,9744,10,1654,3789,2124
2020-07-24,35981,4570,26159,897,880,82,153520,36613,13950,20214,...,299500,1166,19360,14263,415,10093,10,1674,3856,2296


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-22,1190,120,1111,52,33,3,2588,678,133,711,...,45586,34,98,124,0,66,1,458,128,26
2020-07-23,1211,123,1124,52,33,3,2702,688,139,711,...,45639,34,103,129,0,67,1,461,134,28
2020-07-24,1225,128,1136,52,35,3,2807,692,145,711,...,45762,34,106,134,0,70,1,469,136,32


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-22,23924,2463,16983,803,221,58,60531,24766,8770,17849,...,1416,934,9872,7471,365,1950,8,751,1677,510
2020-07-23,24550,2523,17369,803,236,58,62815,25244,8775,17943,...,1425,940,10149,7752,365,2720,8,762,1677,510
2020-07-24,24602,2608,17369,803,241,60,65447,25734,8938,18042,...,1425,946,10472,8127,365,2720,8,779,1677,514


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20
130,84001097,US,USA,840,1097.0,Mobile,Alabama,US,30.784723,-88.208424,...,5349,5514,5713,5899,6017,6150,6279,6443,6797,6901
109,84001055,US,USA,840,1055.0,Etowah,Alabama,US,34.045673,-86.040519,...,1136,1175,1236,1264,1307,1356,1422,1440,1495,1546
110,84001057,US,USA,840,1057.0,Fayette,Alabama,US,33.720769,-87.738866,...,92,104,111,117,119,121,122,124,128,132
111,84001059,US,USA,840,1059.0,Franklin,Alabama,US,34.442353,-87.842895,...,970,979,996,1011,1020,1027,1038,1044,1051,1064
112,84001061,US,USA,840,1061.0,Geneva,Alabama,US,31.093890,-85.835728,...,110,113,118,134,143,146,150,158,167,178


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-22,71813,2131,0,150609,35246,421286,41680,48223,13792,49,...,84417,363615,35578,1366,320,80393,49247,5212,44847,2288
2020-07-23,74212,2193,0,152944,36259,430773,42295,48232,13924,49,...,86987,373037,36099,1377,336,81237,50009,5452,45899,2347
2020-07-24,76005,2249,0,156301,37249,440185,42962,48776,14202,49,...,89078,380364,36962,1385,352,82364,50824,5653,46917,2405


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-22         863    2164     487  282    498     402    693     870   
2020-07-23         877    2423     501  284    528     403    698     908   
2020-07-24         891    2473     507  297    552     406    702     970   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-22          748      174  ...      94      5       40       18   
2020-07-23          756      181  ...      95      5       40       18   
2020-07-24          759      191  ...      97      5       40       25   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-22            201   234   224          0       45      4  
2020-07-23            206   254   227          0       45      4  
2020-07-24            210   264   232          0       45      4  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-22,1364,19,0,2974,380,8047,1771,4406,527,0,...,888,4439,260,56,7,2051,1468,101,865,25
2020-07-23,1397,19,0,3063,386,8201,1786,4410,529,0,...,925,4622,267,56,7,2054,1482,102,878,25
2020-07-24,1438,19,0,3143,394,8337,1790,4413,578,0,...,938,4790,273,56,7,2067,1495,103,878,25


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-22          20      15       4    2      1      11     33       6   
2020-07-23          20      16       4    2      1      11     34       6   
2020-07-24          20      17       4    2      1      11     34       6   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-22           34        7  ...       0      0        0        0   
2020-07-23           34        7  ...       0      0        0        0   
2020-07-24           37        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-22              0     0     0         24        0      0  
2020-07-23              0     0     0         24        0      0  
2020-07-24              0     0     0         24        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-22,0.003145,0.015851,0.024467,0.005656,0.042362,0.000000,0.042478,0.012452,0.031643,0.005144,...,0.001893,0.019161,0.027851,0.030531,0.017456,0.018422,0.0,0.006753,0.058181,0.117582
2020-07-23,0.005626,0.024782,0.024606,0.000000,0.048030,0.000000,0.043178,0.013140,0.022027,0.008530,...,0.002615,0.021486,0.033027,0.034108,0.009804,0.036816,0.0,0.008537,0.057494,0.044248
2020-07-24,0.001475,0.023287,0.026487,0.008999,0.034078,0.078947,0.037108,0.012472,0.026113,0.005722,...,0.002574,0.021911,0.019699,0.047748,0.007282,0.035817,0.0,0.012092,0.017683,0.080979


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-22,0.003373,0.025641,0.010000,0.0,0.100000,0.0,0.039357,0.024169,0.039062,0.001408,...,0.001736,0.030303,0.031579,0.033333,NaN,0.031250,0.0,0.004386,0.000000,0.000000
2020-07-23,0.017647,0.025000,0.011701,0.0,0.000000,0.0,0.044049,0.014749,0.045113,0.000000,...,0.001163,0.000000,0.051020,0.040323,NaN,0.015152,0.0,0.006550,0.046875,0.076923
2020-07-24,0.011561,0.040650,0.010676,0.0,0.060606,0.0,0.038860,0.005814,0.043165,0.000000,...,0.002695,0.000000,0.029126,0.038760,NaN,0.044776,0.0,0.017354,0.014925,0.142857


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-22,0.007708,0.027534,0.020245,0.0,0.000000,0.017544,0.032987,0.023135,0.026331,0.007507,...,0.001414,0.005382,0.036866,0.069884,0.0,0.009317,0.0,0.013495,0.035185,0.045082
2020-07-23,0.026166,0.024361,0.022729,0.0,0.067873,0.000000,0.037733,0.019301,0.000570,0.005266,...,0.006356,0.006424,0.028059,0.037612,0.0,0.394872,0.0,0.014647,0.000000,0.000000
2020-07-24,0.002118,0.033690,0.000000,0.0,0.021186,0.034483,0.041901,0.019411,0.018575,0.005517,...,0.000000,0.006383,0.031826,0.048375,0.0,0.000000,0.0,0.022310,0.000000,0.007843


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-22,0.020680,0.043585,NaN,0.012954,0.017054,0.029272,0.015570,0.002641,0.003346,0.0,...,0.030179,0.035731,0.016166,0.000000,0.052632,0.012876,0.013834,0.007734,0.016132,0.022341
2020-07-23,0.033406,0.029094,NaN,0.015504,0.028741,0.022519,0.014755,0.000187,0.009571,0.0,...,0.030444,0.025912,0.014644,0.008053,0.050000,0.010498,0.015473,0.046048,0.023458,0.025787
2020-07-24,0.024161,0.025536,NaN,0.021949,0.027304,0.021849,0.015770,0.011279,0.019966,0.0,...,0.024038,0.019641,0.023906,0.005810,0.047619,0.013873,0.016297,0.036867,0.022179,0.024712


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-22      0.021302  0.041887  0.006198  0.014388  0.039666  0.015152   
2020-07-23      0.016222  0.119686  0.028747  0.007092  0.060241  0.002488   
2020-07-24      0.015964  0.020636  0.011976  0.045775  0.045455  0.007444   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-07-22      0.007267  0.024735  0.006729  0.011628  ...  0.056180   0.25   
2020-07-23      0.007215  0.043678  0.010695  0.040230  ...  0.010638   0.00   
2020-07-24      0.005731  0.068282  0.003968  0.055249  ...  0.021053   0.00   

Province_State                                                               \
Admin2         Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2020-07-22          0.0  0.000000   0.036082  0.035398  0.004484        NaN   
2020-07-23          0.0  0.000000   0.024876  0.085470  0.013393        NaN   
2020-07-24          0.0  0.388889   0.019417  0.039370  0.022026        NaN   

Province_State                   
Admin2          Washakie Weston  
2020-07-22      0.022727    0.0  
2020-07-23      0.000000    0.0  
2020-07-24      0.000000    0.0  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-22,0.046815,0.055556,NaN,0.019191,0.016043,0.020157,0.004538,0.000000,0.003810,NaN,...,0.019518,0.057156,0.035857,0.0,0.166667,0.001465,0.002048,0.000000,0.006985,0.0
2020-07-23,0.024194,0.000000,NaN,0.029926,0.015789,0.019138,0.008470,0.000908,0.003795,NaN,...,0.041667,0.041226,0.026923,0.0,0.000000,0.001463,0.009537,0.009901,0.015029,0.0
2020-07-24,0.029349,0.000000,NaN,0.026118,0.020725,0.016583,0.002240,0.000680,0.092628,NaN,...,0.014054,0.036348,0.022472,0.0,0.000000,0.006329,0.008772,0.009804,0.000000,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                  \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock    Butler   
2020-07-22         0.0  0.000000     0.0  0.0    0.0     0.1  0.000000   
2020-07-23         0.0  0.066667     0.0  0.0    0.0     0.0  0.030303   
2020-07-24         0.0  0.062500     0.0  0.0    0.0     0.0  0.000000   

Province_State                             ... Wyoming                  \
Admin2         Calhoun  Chambers Cherokee  ...    Park Platte Sheridan   
2020-07-22         0.0  0.030303      0.0  ...     NaN    NaN      NaN   
2020-07-23         0.0  0.000000      0.0  ...     NaN    NaN      NaN   
2020-07-24         0.0  0.088235      0.0  ...     NaN    NaN      NaN   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-22          NaN        NaN   NaN   NaN        0.0      NaN    NaN  
2020-07-23          NaN        NaN   NaN   NaN        0.0      NaN    NaN  
2020-07-24          NaN        NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-22,0.002887,0.020259,0.024917,0.0036,0.044672,0.000427,0.039577,0.010169,0.032121,0.004900,...,0.001883,0.019461,0.032016,0.032967,0.020417,0.029193,5.174014e-11,0.007499,0.057551,0.092068
2020-07-23,0.004257,0.022521,0.024761,0.0018,0.046351,0.000213,0.041378,0.011654,0.027074,0.006715,...,0.002249,0.020474,0.032521,0.033538,0.015110,0.033004,2.587007e-11,0.008018,0.057522,0.068158
2020-07-24,0.002866,0.022904,0.025624,0.0054,0.040214,0.039580,0.039243,0.012063,0.026593,0.006218,...,0.002411,0.021192,0.026110,0.040643,0.011196,0.034411,1.293504e-11,0.010055,0.037603,0.074569


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-22,0.004571,0.025461,0.010187,1.426658e-13,0.059692,1.590048e-30,0.04099,0.020665,0.028725,0.000373,...,0.001637,0.016405,0.040307,0.032838,NaN,0.030645,0.0,0.008595,0.036419,0.008460
2020-07-23,0.011109,0.025231,0.010944,7.133292e-14,0.029846,7.950239e-31,0.04252,0.017707,0.036919,0.000187,...,0.001400,0.008203,0.045664,0.036580,NaN,0.022898,0.0,0.007572,0.041647,0.042691
2020-07-24,0.011335,0.032941,0.010810,3.566646e-14,0.045226,3.975119e-31,0.04069,0.011760,0.040042,0.000093,...,0.002048,0.004101,0.037395,0.037670,NaN,0.033837,0.0,0.012463,0.028286,0.092774


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-22,0.006116,0.023665,0.019170,2.291204e-07,0.014654,0.008862,0.037033,0.022361,0.019188,0.005788,...,0.001211,0.004224,0.025705,0.135340,0.004695,0.025300,3.031649e-13,0.017942,0.031804,0.037575
2020-07-23,0.016141,0.024013,0.020949,1.145602e-07,0.041264,0.004431,0.037383,0.020831,0.009879,0.005527,...,0.003784,0.005324,0.026882,0.086476,0.002348,0.210086,1.515825e-13,0.016294,0.015902,0.018787
2020-07-24,0.009130,0.028851,0.010475,5.728011e-08,0.031225,0.019457,0.039642,0.020121,0.014227,0.005522,...,0.001892,0.005853,0.029354,0.067425,0.001174,0.105043,7.579123e-14,0.019302,0.007951,0.013315


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-22,0.022964,0.043402,NaN,0.016374,0.019318,0.027806,0.013512,0.002036,0.005871,1.000421e-39,...,0.028130,0.030120,0.016023,0.002887,0.03996,0.012807,0.015948,0.013317,0.019339,0.022359
2020-07-23,0.028185,0.036248,NaN,0.015939,0.024030,0.025163,0.014134,0.001112,0.007721,5.002104e-40,...,0.029287,0.028016,0.015333,0.005470,0.04498,0.011653,0.015711,0.029682,0.021398,0.024073
2020-07-24,0.026173,0.030892,NaN,0.018944,0.025667,0.023506,0.014952,0.006195,0.013843,2.501052e-40,...,0.026662,0.023829,0.019620,0.005640,0.04630,0.012763,0.016004,0.033275,0.021789,0.024393


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-22      0.018062  0.045096  0.010366  0.018269  0.039750  0.009575   
2020-07-23      0.017142  0.082391  0.019557  0.012680  0.049996  0.006032   
2020-07-24      0.016553  0.051513  0.015766  0.029228  0.047725  0.006738   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-07-22      0.007549  0.037237  0.006832  0.016214  ...  0.028377   
2020-07-23      0.007382  0.040458  0.008764  0.028222  ...  0.019508   
2020-07-24      0.006556  0.054370  0.006366  0.041735  ...  0.020280   

Province_State                                                              \
Admin2           Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-07-22      0.12500  0.004549  0.024469   0.044906  0.036476  0.004752   
2020-07-23      0.06250  0.002275  0.012235   0.034891  0.060973  0.009072   
2020-07-24      0.03125  0.001137  0.200562   0.027154  0.050171  0.015549   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-07-22            NaN  0.012823  0.012366  
2020-07-23            NaN  0.006411  0.006183  
2020-07-24            NaN  0.003206  0.003092  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-22,0.027495,0.029617,NaN,0.026194,0.018855,0.015763,0.003513,0.000338,0.003008,NaN,...,0.019345,0.040596,0.025432,5.291608e-13,0.083333,0.003469,0.004099,0.002791,0.008177,0.008825
2020-07-23,0.025844,0.014809,NaN,0.028060,0.017322,0.017450,0.005991,0.000623,0.003401,NaN,...,0.030506,0.040911,0.026177,2.645804e-13,0.041667,0.002466,0.006818,0.006346,0.011603,0.004412
2020-07-24,0.027597,0.007404,NaN,0.027089,0.019024,0.017017,0.004115,0.000652,0.048014,NaN,...,0.022280,0.038629,0.024325,1.322902e-13,0.020833,0.004398,0.007795,0.008075,0.005801,0.002206


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                               \
Admin2           Autauga   Baldwin   Barbour      Bibb Blount Bullock   
2020-07-22      0.001796  0.021286  0.042644  0.001953    0.0  0.0500   
2020-07-23      0.000898  0.043976  0.021322  0.000977    0.0  0.0250   
2020-07-24      0.000449  0.053238  0.010661  0.000488    0.0  0.0125   

Province_State                                              ... Wyoming  \
Admin2            Butler   Calhoun  Chambers      Cherokee  ...    Park   
2020-07-22      0.008456  0.000391  0.016313  1.669780e-10  ...     NaN   
2020-07-23      0.019379  0.000195  0.008156  8.348902e-11  ...     NaN   
2020-07-24      0.009690  0.000098  0.048196  4.174451e-11  ...     NaN   

Province_State                                                             \
Admin2         Platte Sheridan Sublette Sweetwater Teton Uinta Unassigned   
2020-07-22        NaN      NaN      NaN        NaN   NaN   NaN   0.009221   
2020-07-23        NaN      NaN      NaN        NaN   NaN   NaN   0.004611   
2020-07-24        NaN      NaN      NaN        NaN   NaN   NaN   0.002305   

Province_State                  
Admin2         Washakie Weston  
2020-07-22          NaN    NaN  
2020-07-23          NaN    NaN  
2020-07-24          NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200724,AK,2748,197824,NaN,37.0,NaN,NaN,NaN,1.0,...,200572,0,0,4ed800f28080931e4bb0737b0dbcad613b757035,0,0,0,0,0,NaN
1,20200724,AL,76005,552979,NaN,1570.0,9157.0,NaN,1058.0,NaN,...,628984,41,162,4718e0aea8aa7c100fd571642ec4abcb57b02fed,0,0,0,0,0,NaN
2,20200724,AR,37249,424215,NaN,497.0,2361.0,NaN,NaN,109.0,...,461464,8,0,743c3a7f26ca7cf1ce0f7a6aedb4d935d2840ab9,0,0,0,0,0,NaN
3,20200724,AS,0,1037,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,f959d1a6b8d4b3f47c048f022a17cfc45bac67e4,0,0,0,0,0,NaN
4,20200724,AZ,156301,680516,NaN,2844.0,7461.0,837.0,NaN,575.0,...,836817,79,225,e16d3bbbbdab36942336b77d242ead15a4ff0c1b,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200724,AK,2748.0,197824.0,NaN,37.0,NaN,NaN,NaN,1.0,...,200572,0,0,4ed800f28080931e4bb0737b0dbcad613b757035,0,0,0,0,0,NaN
1,20200724,AL,76005.0,552979.0,NaN,1570.0,9157.0,NaN,1058.0,NaN,...,628984,41,162,4718e0aea8aa7c100fd571642ec4abcb57b02fed,0,0,0,0,0,NaN
2,20200724,AR,37249.0,424215.0,NaN,497.0,2361.0,NaN,NaN,109.0,...,461464,8,0,743c3a7f26ca7cf1ce0f7a6aedb4d935d2840ab9,0,0,0,0,0,NaN
3,20200724,AS,0.0,1037.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,f959d1a6b8d4b3f47c048f022a17cfc45bac67e4,0,0,0,0,0,NaN
4,20200724,AZ,156301.0,680516.0,NaN,2844.0,7461.0,837.0,NaN,575.0,...,836817,79,225,e16d3bbbbdab36942336b77d242ead15a4ff0c1b,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-24,AK,2748,197824,NaN,37.0,NaN,NaN,NaN,1.0,NaN,...,200572,0,0,4ed800f28080931e4bb0737b0dbcad613b757035,0,0,0,0,0,NaN
2020-07-24,AL,76005,552979,NaN,1570.0,9157.0,NaN,1058.0,NaN,565.0,...,628984,41,162,4718e0aea8aa7c100fd571642ec4abcb57b02fed,0,0,0,0,0,NaN
2020-07-24,AR,37249,424215,NaN,497.0,2361.0,NaN,NaN,109.0,329.0,...,461464,8,0,743c3a7f26ca7cf1ce0f7a6aedb4d935d2840ab9,0,0,0,0,0,NaN
2020-07-24,AS,0,1037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,f959d1a6b8d4b3f47c048f022a17cfc45bac67e4,0,0,0,0,0,NaN
2020-07-24,AZ,156301,680516,NaN,2844.0,7461.0,837.0,NaN,575.0,NaN,...,836817,79,225,e16d3bbbbdab36942336b77d242ead15a4ff0c1b,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-24,AK,2748.0,197824.0,NaN,37.0,NaN,NaN,NaN,1.0,NaN,...,200572,0,0,4ed800f28080931e4bb0737b0dbcad613b757035,0,0,0,0,0,NaN
2020-07-24,AL,76005.0,552979.0,NaN,1570.0,9157.0,NaN,1058.0,NaN,565.0,...,628984,41,162,4718e0aea8aa7c100fd571642ec4abcb57b02fed,0,0,0,0,0,NaN
2020-07-24,AR,37249.0,424215.0,NaN,497.0,2361.0,NaN,NaN,109.0,329.0,...,461464,8,0,743c3a7f26ca7cf1ce0f7a6aedb4d935d2840ab9,0,0,0,0,0,NaN
2020-07-24,AS,0.0,1037.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,f959d1a6b8d4b3f47c048f022a17cfc45bac67e4,0,0,0,0,0,NaN
2020-07-24,AZ,156301.0,680516.0,NaN,2844.0,7461.0,837.0,NaN,575.0,NaN,...,836817,79,225,e16d3bbbbdab36942336b77d242ead15a4ff0c1b,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE